In [1]:
import os, sys
import pandas as pd
import numpy as np
import torch
import torch.optim as optim

from torch.utils.data import DataLoader

In [4]:
sys.path.append('..')
from src.model.handler import ModelHandler
from src.data.dataset import GWDataset, Normalize
from src.model.mlp import MLP

In [5]:
base_data_dir = '/srv/scratch/z5370003/projects/data'
data_dir = os.path.join(base_data_dir, 
                        'groundwater/FEFLOW/coastal/',
                         'variable_density/')
data_path = os.path.join(data_dir, 'filter')

In [6]:
_mean = np.array([1523.10199582, 1729.62046887,   26.72577967,  316.5513978])
_std = np.array([569.13566635, 566.33636362,  14.90088159, 183.2160048 ])
input_transform = Normalize(mean=_mean, std=_std)

In [7]:
_mean = np.array([3.29223762e-01, 1.78798090e+04])
_std = np.array([2.17284490e-01, 1.51990336e+04])
output_transform = Normalize(mean=_mean, std=_std)

In [8]:
train_ds = GWDataset(data_path, 'train', 
                     input_transform=input_transform, 
                     output_transform=output_transform)
len(train_ds)

6831636

In [9]:
train_dl = DataLoader(train_ds, batch_size=4096, 
                      shuffle=True, pin_memory=True)
len(train_dl)

1668

In [10]:
test_ds = GWDataset(data_path, 'val', 
                     input_transform=input_transform, 
                     output_transform=output_transform)


test_dl = DataLoader(test_ds, batch_size=4096, 
                      shuffle=False, pin_memory=True)
len(test_dl)

717

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
input_size = 4
hidden_sizes = [32, 64, 128, 32, 128, 32]
output_size = 2

mlp = MLP(input_size=input_size,
          hidden_sizes=hidden_sizes,
          output_size=output_size)

opt = optim.Adam(mlp.parameters(), lr=1e-3, weight_decay=1e-3)

In [14]:
from torchsummary import summary
print(summary(mlp, (1, input_size)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 32]             160
              ReLU-2                [-1, 1, 32]               0
            Linear-3                [-1, 1, 64]           2,112
              ReLU-4                [-1, 1, 64]               0
            Linear-5               [-1, 1, 128]           8,320
              ReLU-6               [-1, 1, 128]               0
            Linear-7                [-1, 1, 32]           4,128
              ReLU-8                [-1, 1, 32]               0
            Linear-9               [-1, 1, 128]           4,224
             ReLU-10               [-1, 1, 128]               0
           Linear-11                [-1, 1, 32]           4,128
             ReLU-12                [-1, 1, 32]               0
           Linear-13                 [-1, 1, 2]              66
Total params: 23,138
Trainable params: 

In [11]:
model_handler = ModelHandler(model=mlp, device=device, optimizer=opt)

In [12]:
model_handler.train(train_dl, num_epochs=50)

 11%|█         | 101/954 [02:10<18:23,  1.29s/it]


KeyboardInterrupt: 

In [ ]:
print(f"Train loss: {model_handler.evaluate(train_dl)}")
print(f"Test loss: {model_handler.evaluate(test_dl)}")